---
**License**

 VexpBounce

 Tue Nov 15 12:13:14 2022\
 Copyright  2022\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 MagDustBounce\
 Copyright (C) 2022 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

# Magnetic field in dust dominated bouncing cosmology

In this notebook we develop the necessary objects to compute the power spectrum (and other observables) for magnetic field production in the contraction phase of a bouncing cosmology dominated by a scalar field.

The scale factor for this model is described at 1706.08830.

The electromagnetic vector field satisfy the following Hamilton equations:
\begin{equation}
\dot{A} = \frac{\Pi_A}{m}, \qquad \dot{\Pi}_A = -m\nu^2 A, \qquad m \equiv \frac{a F}{N},\quad \nu = \frac{Nk}{a},\qquad N = \frac{\mathrm{d}t}{\mathrm{d}\tau}, \qquad \alpha = \alpha_b + \frac{\tau^2}{2},\qquad \alpha = \ln(a), \qquad N = \frac{\tau}{H}.
\end{equation}
for the mode $k$. It is convenient to rewrite $F$ as
\begin{equation}
F\equiv \frac{1}{4} + e^{\alpha^2-\frac{\phi^2}{\beta^2}}.
\end{equation}
The coupling is controlled by the parameters $\phi_0$ and $\beta$.

## Loading NumCosmo

The first step is to load both NumCosmo and NumCosmoMath libraries.

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import RegularGridInterpolator

from numcosmo_py import Ncm, Nc
from numcosmo_py.plotting.tools import set_rc_params_article, latex_float

# Initializing the library

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

figures_dir = "path-to-figures"


# Latex definition

In [ ]:
import matplotlib.pylab as pylab

# params = {
#    "legend.fontsize": "x-large",
#    "figure.figsize": (15, 10),
#    "axes.labelsize": "24",
#    "axes.titlesize": "x-large",
#    "xtick.labelsize": "26",
#    "ytick.labelsize": "26",
# }
# pylab.rcParams.update(params)

# Model initialization

Creating a new background Vexp model

In [ ]:
Vexp_none = Nc.HICosmoVexp.new()
Vexp_none.set_em_coupling(Nc.HICosmoVexpEMCoupling.NONE)

Vexp_cauchy = Nc.HICosmoVexp.new()
Vexp_cauchy.set_em_coupling(Nc.HICosmoVexpEMCoupling.CAUCHY)

Vexp_gauss = Nc.HICosmoVexp.new()
Vexp_gauss.set_em_coupling(Nc.HICosmoVexpEMCoupling.GAUSS)

use_set1 = True

if use_set1:
    current_set = {
        "alphab": 8.3163e-2,  # Alpha (# of e-fold\s) at the bounce
        "sigmaphi": 9.0,  # Width of the Gaussian solution for the WdW equation
        "xb": 2.0e36,  # Inverse of the scale factor at the bounce (Initial condition)
        "dphi": -9.0e-4,  # Deviation of the Gaussian solution for the WdW equation
        "OmegaL": 1.0,  # H²(a when w=-1)/H²(a0). Basically gives the DE-dominated phase
        "Omegac": 1.0,  # Omega_d???
        "H0": 67.8,  # Hubble parameter today given by CMB observations
        "alphaem": 14.4,  # Amplitude of the EM gaussian coupling
        "betaem": 2.2,  # Width of the EM gaussian coupling
    }
    currentset = "Set1"
else:
    current_set = {
        "alphab": 7.4847e-3,  # Alpha (# of e-fold\s) at the bounce
        "sigmaphi": 100.0,  # Width of the Gaussian solution for the WdW equation
        "xb": 4.0e36,  # Inverse of the scale factor at the bounce (Initial condition)
        "dphi": -9.0e-4,  # Deviation of the Gaussian solution for the WdW equation
        "OmegaL": 1.0,  # H²(a when w=-1)/H²(a0). Basically gives the DE-dominated phase
        "Omegac": 1.0,  # Omega_d???
        "H0": 67.8,  # Hubble parameter today given by CMB observations
        "alphaem": 13.6,  # Amplitude of the EM gaussian coupling
        "betaem": 2.9,  # Width of the EM gaussian coupling
    }
    currentset = "Set2"

Vexp_none.set_properties(**current_set)
Vexp_cauchy.set_properties(**current_set)
Vexp_gauss.set_properties(**current_set)

tc = Vexp_none.tau_xe(
    1.0e15
)  # Initial tau in expansion phase??? tau para quando a expansão é 10^15
reltol = 1.0e-14  # Relative tolerance for solving differential equations

Vexp_none.tau_xe(1.0)

# Model integrated time

Getting the time intervals for the computed model in terms of $\alpha \equiv \ln(a/a_0)$ and $\alpha = \alpha_b + \frac{\tau^2}{2}$.



In [ ]:
tau_min = Vexp_none.tau_min()  # Get initial time
tau_max = Vexp_none.tau_max()  # Get final time
tau_qt_c = Vexp_none.tau_qt_c()  # Get initial time of the quantum phase
tau_qt_e = Vexp_none.tau_qt_e()  # Get final time of the quantum phase
npoints = 5000

tau_a = np.linspace(tau_min, tau_max, npoints)
tau_q_a = np.linspace(tau_qt_c, tau_qt_e, npoints)

print(f"Vexp model prepared in the interval ({tau_min}, {tau_max})")
print(f"Vexp quantum dominated interval ({tau_qt_c}, {tau_qt_e})")

Computing background functions

In [ ]:
alpha_a = [Vexp_none.alpha(tau) for tau in tau_a]
phi_a = np.array([Vexp_none.phi(tau) for tau in tau_a])
deta_s = Ncm.SplineCubicNotaknot.new()
deta_s.set_array(tau_a, [Vexp_none.eval_nu(tau, 1.0) for tau in tau_a], True)
rescale = Vexp_none.RH_planck() / np.exp(Vexp_none.alpha_0e())
eta_a = [
    (
        -deta_s.eval_integ(tau, 0.0) * rescale
        if tau < 1.0
        else deta_s.eval_integ(0.0, tau) * rescale
    )
    for tau in tau_a
]
RS_a = np.array(
    [Vexp_none.Ricci_scale(tau) for tau in tau_a]
)  # Hubble scale goes to 0 at the bounce, but not the Ricci scalar
nu_a = np.array([Vexp_none.eval_nu(tau, 1.0) for tau in tau_a])
x_y_a = np.array([Vexp_none.x_y(tau) for tau in tau_a])

# Consistence check

Plotting $\alpha(\tau) - \alpha_b$.

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

plt.plot(tau_a, alpha_a, label=r"$\alpha - \alpha_b$")
plt.axvline(x=Vexp_none.tau_xc(1.0), color="green", linestyle="--")
plt.axvline(x=Vexp_none.tau_xe(1.0), color="black", linestyle="--")

plt.text(
    Vexp_none.tau_xc(1.0) + 1.0,
    100.0,
    rf"$\vert H(\tau = {Vexp_none.tau_xc(1.0):.2f})\vert = H_0$",
    color="green",
    ha="left",
    va="bottom",
)
plt.text(
    Vexp_none.tau_xe(1.0) - 1.0,
    180.0,
    rf"$\vert H(\tau = {Vexp_none.tau_xe(1.0):.2f})\vert = H_0$",
    color="black",
    ha="right",
    va="bottom",
)

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best")

plt.title(rf"Log-scale factor: $\ln(a(\tau)/a_b)$")
plt.xlabel(r"$\tau$")
plt.xlim(tau_min, 10.0)
plt.savefig(f"{figures_dir}/alpha.pdf")
plt.show()

# Plotting scale factor as a function of tau.

$a(\tau)$

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

print(Vexp_none.alpha_0e())  # alpha_0 na expansão
print(Vexp_none.tau_xe(1.0))

plt.plot(tau_a, np.exp(alpha_a) / Vexp_none.props.xb, label=r"$a$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.xlabel(r"$\tau$")
plt.yscale("log")
plt.show()
pass

# Plotting the scalar field as a function of tau.

$\phi(\tau)$

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

plt.plot(tau_a, phi_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.xlabel(r"$\tau$")
plt.xlim(tau_min, 10.0)
plt.savefig(f"{figures_dir}/phi.pdf", bbox_inches="tight")
plt.show()

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

plt.plot(eta_a, phi_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.xscale("symlog", linthresh=1.0e-1)
plt.xlabel(r"$\eta$")
plt.show()

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

# Vexp_cauchy.set_properties(betaem=3.0e0, phiem0=40.0)
# Vexp_gauss.set_properties(betaem=3.0e0, phiem0=40.0)

m_cauchy_a = np.array([Nc.HIPertIEM.eval_m(Vexp_cauchy, tau, 1.0) for tau in tau_a])
m_gauss_a = np.array([Nc.HIPertIEM.eval_m(Vexp_gauss, tau, 1.0) for tau in tau_a])
xi_cauchy_a = np.log(m_cauchy_a * nu_a)
xi_gauss_a = np.log(m_gauss_a * nu_a)

plt.plot(tau_a, xi_cauchy_a, label=r"$m_\mathrm{C}$")
plt.plot(tau_a, xi_gauss_a, label=r"$m_\mathrm{G}$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)
plt.xlim(None, 9.3)

# plt.yscale("symlog", linthresh=1.0e-2)
plt.xlabel(r"$\tau$")
plt.savefig(f"{figures_dir}/masses.pdf", bbox_inches="tight")
plt.show()

# Ploting Ricci scalar and mode frequency.

$R(\tau)$ and $\nu(\tau)$

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

plt.plot(tau_a, RS_a, label=r"$R_s$")
plt.plot(tau_a, nu_a**2, label=r"$\nu^2$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.xlabel(r"$\tau$")
plt.xlim(tau_min, 9.2)
plt.yscale("log")
plt.savefig(f"{figures_dir}/ricci_nu.pdf", bbox_inches="tight")
plt.show()

# Plotting background field variables

$x(\tau)$, $y(\tau)$ and $\sqrt{x^2 + y^2}$.

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

plt.plot(tau_a, x_y_a[:, 0], label="$x$")
plt.plot(tau_a, x_y_a[:, 1], label="$y$")
plt.plot(tau_a, np.hypot(x_y_a[:, 0], x_y_a[:, 1]), label=r"$\sqrt{x^2+y^2}$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.xlabel(r"$\tau$")
plt.xlim(tau_min, 9.2)
plt.yscale("symlog")

plt.savefig(f"{figures_dir}/xy.pdf", bbox_inches="tight")

plt.show()

# Close-up on the Friedmann constraint in the quantum regime
plt.figure()

plt.plot(
    tau_a,
    np.sqrt(x_y_a[:, 0] * x_y_a[:, 0] + x_y_a[:, 1] * x_y_a[:, 1]),
    label="$x^2+y^2$",
)
plt.plot(
    tau_a,
    np.hypot(x_y_a[:, 0], x_y_a[:, 1]),
    label=r"$\sqrt{x^2+y^2}$",
    linestyle="dashed",
)

plt.xlim(-0.25, 0.25)
plt.yscale("symlog")

plt.show()

# Plotting magnetic field background quantities 

$\phi(\tau)$, $f(\phi(\tau)) = e^{-\phi^2/\beta^2}$ and the equation of state $w$.

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

plt.plot(tau_a, phi_a, label=r"$\phi$")
# plt.plot (tau_a, np.exp (-(phi_a / beta)**2), label=r'$f(\phi)$')
plt.plot(tau_a, 2.0 * x_y_a[:, 0] ** 2 - 1.0, label="$w = 2x^2-1$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.xlabel(r"$\tau$")
# plt.yscale('symlog')
plt.xlim(-2.0, tau_qt_e)
plt.ylim(-1.1, 1.1)

plt.show()

###
### Equation of state
###

plt.figure()

plt.plot(tau_a, 2.0 * x_y_a[:, 0] ** 2 - 1.0, label="$w = 2x^2-1$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)
plt.xlim(-0.025, 0.025)
plt.xlabel(r"$\tau$")

plt.show()

# Defines the electromagnetic field model

This model use the Complex Structure Quantization method CSQ1D.

In [ ]:
tau_f = 9.3
# tau_f = 9.448 # Set 1
# tau_f = 9.456 # Set 2

pem_none = Nc.HIPertEM.new()
pem_none.set_initial_condition_type(Ncm.CSQ1DInitialStateType.ADIABATIC2)
pem_none.set_abstol(
    1.0e-200
)  # The vacuum is a fixed point at zero, we need an absolute tolerance to evolve the system
pem_none.set_k(4.0e3)
pem_none.set_ti(Vexp_none.tau_min())
pem_none.set_tf(tau_f)  # We do not want to evolve through the singularity
pem_none.set_vacuum_max_time(-12.0)
pem_none.set_vacuum_reltol(1.0e-8)


pem_cauchy = Nc.HIPertEM.new()
pem_cauchy.set_k(4.0e3)
pem_cauchy.set_initial_condition_type(Ncm.CSQ1DInitialStateType.ADIABATIC4)
pem_cauchy.set_ti(Vexp_cauchy.tau_min())
pem_cauchy.set_tf(tau_f)  # We do not want to evolve through the singularity
pem_cauchy.set_vacuum_max_time(-1.0e-1)
pem_cauchy.set_vacuum_reltol(1.0e-8)


pem_gauss = Nc.HIPertEM.new()
pem_gauss.set_initial_condition_type(Ncm.CSQ1DInitialStateType.ADIABATIC2)
pem_gauss.set_k(4.0e3)
pem_gauss.set_ti(Vexp_gauss.tau_min())
pem_gauss.set_tf(tau_f)  # We do not want to evolve through the singularity
pem_gauss.set_vacuum_max_time(-1.0e-1)
pem_gauss.set_vacuum_reltol(1.0e-8)

# Setting up models

Setting up all models, including the benchmark with no coupling.

# Finding starting point as the end of the adiabatic phase

Setting limit for the adiabatic phase with $10^{-9}$.

In [ ]:
print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

(Found_tau_i_none, tau_i_none) = pem_none.find_adiab_time_limit(
    Vexp_none, tau_min, -12.0, 1.0e-9
)
(Found_tau_i_cauchy, tau_i_cauchy) = pem_cauchy.find_adiab_time_limit(
    Vexp_cauchy, tau_min, -1.0e-5, 1.0e-9
)
(Found_tau_i_gauss, tau_i_gauss) = pem_gauss.find_adiab_time_limit(
    Vexp_gauss, tau_min, -1.0e-5, 1.0e-9
)

# Checking if the search for the starting time was successful 

In [ ]:
print(
    f"Found ({Found_tau_i_none  }) WKB scale for no-potential at     {tau_i_none:7.3f}"
)
print(
    f"Found ({Found_tau_i_cauchy}) WKB scale for cauchy-potential at {tau_i_cauchy:7.3f} (alpha = {Vexp_gauss.props.alphaem}, beta = {Vexp_gauss.props.betaem})"
)
print(
    f"Found ({Found_tau_i_gauss }) WKB scale for gauss-potential at  {tau_i_gauss:7.3f} (alpha = {Vexp_gauss.props.alphaem}, beta = {Vexp_gauss.props.betaem})"
)

print(pem_none.eval_F1(Vexp_none, tau_i_none))
print(pem_none.eval_F2(Vexp_none, tau_i_none))
print(pem_cauchy.eval_F1(Vexp_cauchy, tau_i_cauchy))
print(pem_cauchy.eval_F2(Vexp_cauchy, tau_i_cauchy))
print(pem_gauss.eval_F1(Vexp_gauss, tau_i_gauss))
print(pem_gauss.eval_F2(Vexp_gauss, tau_i_gauss))

state_none = Ncm.CSQ1DState.new()
state_cauchy = Ncm.CSQ1DState.new()
state_gauss = Ncm.CSQ1DState.new()

print(pem_none.compute_adiab(Vexp_none, tau_i_none, state_none)[0].get_ag())
print(pem_cauchy.compute_adiab(Vexp_cauchy, tau_i_cauchy, state_cauchy)[0].get_ag())
print(pem_gauss.compute_adiab(Vexp_gauss, tau_i_gauss, state_gauss)[0].get_ag())

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

tau_ad_a = np.linspace(-2.35, -1.0, 100)
tau_ad_a = tau_a

F1_a_cauchy = np.abs([pem_cauchy.eval_F1(Vexp_cauchy, tau) for tau in tau_ad_a])
F2_a_cauchy = np.abs([pem_cauchy.eval_F2(Vexp_cauchy, tau) for tau in tau_ad_a])

F1_a_gauss = np.abs([pem_gauss.eval_F1(Vexp_gauss, tau) for tau in tau_ad_a])
F2_a_gauss = np.abs([pem_gauss.eval_F2(Vexp_gauss, tau) for tau in tau_ad_a])

plt.plot(tau_ad_a, F1_a_cauchy, label=r"cauchy $|F_1|$")
plt.plot(tau_ad_a, F1_a_gauss, label=r"gauss  $|F_1|$")

plt.plot(tau_ad_a, F2_a_cauchy, label=r"cauchy $|F_2|$")
plt.plot(tau_ad_a, F2_a_gauss, label=r"gauss  $|F_2|$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2)

plt.ylim(1.0e-20, None)
plt.xlabel(r"$\tau$")
plt.yscale("log")

plt.show()

In [ ]:
def compute_PE_PB(pem, Vexp, min_tau=None):
    state = Ncm.CSQ1DState.new()
    pem.prepare(Vexp)
    tau_evol = np.array(pem.get_time_array()[0])
    if min_tau is not None:
        tau_evol = tau_evol[tau_evol > min_tau]

    PE_a, PB_a, J11_a, J22_a = [], [], [], []
    for tau in tau_evol:
        PE, PB = pem.eval_PE_PB(Vexp, tau)
        J11, _, J22 = pem.eval_at(Vexp, tau, state).get_J()
        PE_a.append(PE)
        PB_a.append(PB)
        J11_a.append(J11)
        J22_a.append(J22)

    return tau_evol, np.array(PE_a), np.array(PB_a), np.array(J11_a), np.array(J22_a)

In [ ]:
def compute_and_plot_PE_PB(pem, Vexp, label, ax, mylw=1):
    tau_a, PE_a, PB_a, J11_a, J22_a = compute_PE_PB(pem, Vexp)

    if ax[0] is not None:
        ax[0].plot(
            tau_a, PE_a, lw=mylw, label=f"{label} $P_{{E}}(k_s = {pem.get_k()})$"
        )
    if ax[1] is not None:
        ax[1].plot(
            tau_a,
            PB_a,
            lw=mylw,
            label=rf"{label} $P_{{B}}(k_s = {pem.get_k()})$",
            linestyle="dashed",
        )
    if ax[2] is not None:
        ax[2].plot(
            tau_a,
            J11_a,
            lw=mylw,
            label=rf"{label} $J_{{11}}(k_s = {pem.get_k()})$",
            linestyle="dashed",
        )
    if ax[3] is not None:
        ax[3].plot(
            tau_a, J22_a, lw=mylw, label=rf"{label} $J_{{22}}(k_s = {pem.get_k()})$"
        )

In [ ]:
def compute_and_plot_E_B(pem, Vexp, label, ax, mylw=1):
    tau_a, PE_a, PB_a, _, _ = compute_PE_PB(pem, Vexp)

    if ax[0] is not None:
        ax[0].plot(
            tau_a,
            np.sqrt(PE_a),
            lw=mylw,
            label=rf"{label} $E_{{\lambda}}(k_s = {pem.get_k()})$",
        )
    if ax[1] is not None:
        ax[1].plot(tau_a, np.sqrt(PB_a), lw=mylw, label=rf"$B_{{\lambda}}$")

In [ ]:
def compute_f_E_B(pem, Vexp, alpha, beta):
    Vexp.set_properties(alphaem=alpha, betaem=beta)
    _, _, PB_a, _, _ = compute_PE_PB(pem, Vexp)
    return np.sqrt(PB_a)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

set_rc_params_article(column_width=280.0, ncol=2, nrows=0.5)
fig1, axs1 = plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={"wspace": 0.5})
fig2, axs2 = plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={"wspace": 0.5})
fig3, axs3 = plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={"wspace": 0.5})
fig4, axs4 = plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={"wspace": 0.5})

min_tau = -12.0

for i, alpha in enumerate([13.15, 14.4]):

    n = 3
    beta_start = 1.0
    beta_end = 2.98
    phi0_start = alpha * beta_start
    phi0_end = alpha * beta_end

    colorscauchy = plt.cm.autumn(np.linspace(0, 0.8, n))
    colorsgauss = plt.cm.Blues(np.linspace(0.3, 0.8, n))

    Vexp_none.set_properties(betaem=beta_start, alphaem=alpha)
    tau_none_a, PE_none_a, PB_none_a, J11_none_a, J22_none_a = compute_PE_PB(
        pem_none, Vexp_none, min_tau=min_tau
    )
    
    axs1[i].set_yscale("log")
    axs2[i].set_yscale("log")
    axs3[i].set_yscale("log")
    axs4[i].set_yscale("log")
    
    # Main plot axes
    axs1[i].plot(tau_none_a, PE_none_a, lw=1, label=f"no coupling", color="black")
    axs2[i].plot(tau_none_a, PB_none_a, lw=1, label=f"no coupling", color="black")
    axs3[i].plot(tau_none_a, J11_none_a, lw=1, label=f"no coupling", color="black")
    axs4[i].plot(tau_none_a, J22_none_a, lw=1, label=f"no coupling", color="black")
    
    # Zoomed-in region (tau > 9.2)
    inset_width = 0.3
    inset_ax1 = inset_axes(axs1[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.5, inset_width, 0.5),
                   bbox_transform=axs1[i].transAxes, borderpad=0)
    inset_ax1b = inset_axes(axs1[i], width="100%", height="100%",
                   bbox_to_anchor=(1.0, 0.0, inset_width, 0.5),
                   bbox_transform=axs1[i].transAxes, borderpad=0)
    inset_ax2 = inset_axes(axs2[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.5, inset_width, 0.5),
                   bbox_transform=axs2[i].transAxes, borderpad=0)
    inset_ax2b = inset_axes(axs2[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.0, inset_width, 0.5),
                   bbox_transform=axs2[i].transAxes, borderpad=0)
    inset_ax3 = inset_axes(axs3[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.5, inset_width, 0.5),
                   bbox_transform=axs3[i].transAxes, borderpad=0)
    inset_ax3b = inset_axes(axs3[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.0, inset_width, 0.5),
                   bbox_transform=axs3[i].transAxes, borderpad=0)
    inset_ax4 = inset_axes(axs4[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.5, inset_width, 0.5),
                   bbox_transform=axs4[i].transAxes, borderpad=0)
    inset_ax4b = inset_axes(axs4[i], width="100%", height="100%",
                   bbox_to_anchor=(1., 0.0, inset_width, 0.5),
                   bbox_transform=axs4[i].transAxes, borderpad=0)
     
    inset_ax1.set_yscale("log")
    inset_ax1b.set_yscale("log")
    inset_ax2.set_yscale("log")
    inset_ax2b.set_yscale("log")
    inset_ax3.set_yscale("log")
    inset_ax3b.set_yscale("log")
    inset_ax4.set_yscale("log")
    inset_ax4b.set_yscale("log")

    print(f"Main plot {i} y-limits before inset creation: {axs1[i].get_ylim()}")
    # Adjust the range of the inset
    zoom_start = 8.9
    zoom_end = 9.3

    tau_zoom_none_a = (tau_none_a > zoom_start) & (tau_none_a < zoom_end)
    
    # Plot the same data in the inset
    # inset_ax1b.plot(tau_none_a[tau_zoom_none_a], PE_none_a[tau_zoom_none_a], lw=1, label="no coupling", color="black")
    # inset_ax2b.plot(tau_none_a[tau_zoom_none_a], PB_none_a[tau_zoom_none_a], lw=1, label="no coupling", color="black")
    # inset_ax3b.plot(tau_none_a[tau_zoom_none_a], J11_none_a[tau_zoom_none_a], lw=1, label="no coupling", color="black")
    # inset_ax4b.plot(tau_none_a[tau_zoom_none_a], J22_none_a[tau_zoom_none_a], lw=1, label="no coupling", color="black")

    
    for j, phi0 in enumerate(np.linspace(phi0_start, phi0_end, n)):
        beta = phi0 / alpha

        alphaC = alpha
        betaC = beta * alphaC * np.exp(-(alphaC**2) / 2.0)

        Vexp_cauchy.set_properties(betaem=betaC, alphaem=alphaC)
        Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)

        tau_cauchy_a, PE_cauchy_a, PB_cauchy_a, J11_cauchy_a, J22_cauchy_a = (
            compute_PE_PB(pem_cauchy, Vexp_cauchy, min_tau=min_tau)
        )

        tau_gauss_a, PE_gauss_a, PB_gauss_a, J11_gauss_a, J22_gauss_a = compute_PE_PB(
            pem_gauss, Vexp_gauss, min_tau=min_tau
        )
        tau_zoom_cauchy_a = (tau_cauchy_a > zoom_start) & (tau_cauchy_a < zoom_end)
        tau_zoom_gauss_a = (tau_gauss_a > zoom_start) & (tau_gauss_a < zoom_end)

        axs1[i].plot(
            tau_cauchy_a,
            PE_cauchy_a,
            lw=1,
            label=rf"$\beta_\mathrm{{C}}$ = {beta:.2f}",
            color=colorscauchy[j],
            linestyle="dashed",
        )
        axs1[i].plot(
            tau_gauss_a,
            PE_gauss_a,
            lw=1,
            label=rf"$\beta_\mathrm{{G}}$ = {beta:.2f}",
            color=colorsgauss[j],
        )

        axs2[i].plot(
            tau_cauchy_a,
            PB_cauchy_a,
            lw=1,
            label=rf"$\beta_\mathrm{{C}}$ = {beta:.2f}",
            color=colorscauchy[j],
            linestyle="dashed",
        )
        axs2[i].plot(
            tau_gauss_a,
            PB_gauss_a,
            lw=1,
            label=rf"$\beta_\mathrm{{G}}$ = {beta:.2f}",
            color=colorsgauss[j],
        )

        axs3[i].plot(
            tau_cauchy_a,
            0.5 * J11_cauchy_a,
            lw=1,
            label=rf"$\beta_\mathrm{{C}}$ = {beta:.2f}",
            color=colorscauchy[j],
            linestyle="dashed",
        )
        axs3[i].plot(
            tau_gauss_a,
            0.5 * J11_gauss_a,
            lw=1,
            label=rf"$\beta_\mathrm{{G}}$ = {beta:.2f}",
            color=colorsgauss[j],
        )

        axs4[i].plot(
            tau_cauchy_a,
            0.5 * J22_cauchy_a,
            lw=1,
            label=rf"$\beta_\mathrm{{C}}$ = {beta:.2f}",
            color=colorscauchy[j],
            linestyle="dashed",
        )
        axs4[i].plot(
            tau_gauss_a,
            0.5 * J22_gauss_a,
            lw=1,
            label=rf"$\beta_\mathrm{{G}}$ = {beta:.2f}",
            color=colorsgauss[j],
        )
        
        # Inset
        # Plot on the inset axes for the zoomed-in region
        inset_ax1b.plot(
            tau_cauchy_a[tau_zoom_cauchy_a],
            PE_cauchy_a[tau_zoom_cauchy_a],
            lw=1,
            color=colorscauchy[j],
            linestyle="dashed",
        )
        inset_ax1.plot(
            tau_gauss_a[tau_zoom_gauss_a],
            PE_gauss_a[tau_zoom_gauss_a],
            lw=1,
            color=colorsgauss[j],
        )

        inset_ax2b.plot(
            tau_cauchy_a[tau_zoom_cauchy_a],
            PB_cauchy_a[tau_zoom_cauchy_a],
            lw=1,
            color=colorscauchy[j],
            linestyle="dashed",
        )
        inset_ax2.plot(
            tau_gauss_a[tau_zoom_gauss_a],
            PB_gauss_a[tau_zoom_gauss_a],
            lw=1,
            color=colorsgauss[j],
        )

        inset_ax3b.plot(
            tau_cauchy_a[tau_zoom_cauchy_a],
            0.5 * J11_cauchy_a[tau_zoom_cauchy_a],
            lw=1,
            color=colorscauchy[j],
            linestyle="dashed",
        )
        inset_ax3.plot(
            tau_gauss_a[tau_zoom_gauss_a],
            0.5 * J11_gauss_a[tau_zoom_gauss_a],
            lw=1,
            color=colorsgauss[j],
        )

        inset_ax4b.plot(
            tau_cauchy_a[tau_zoom_cauchy_a],
            0.5 * J22_cauchy_a[tau_zoom_cauchy_a],
            lw=1,
            color=colorscauchy[j],
            linestyle="dashed",
        )
        inset_ax4.plot(
            tau_gauss_a[tau_zoom_gauss_a],
            0.5 * J22_gauss_a[tau_zoom_gauss_a],
            lw=1,
            color=colorsgauss[j],
        )

    alpha_val = str(alpha)

    axs1[i].set_xlabel(r"$\tau$")
    axs2[i].set_xlabel(r"$\tau$")
    axs3[i].set_xlabel(r"$\tau$")
    axs4[i].set_xlabel(r"$\tau$")

    if i == 0:
        axs1[i].set_ylabel(r"$P_{E}$")
        axs2[i].set_ylabel(r"$P_{B}$")
        axs3[i].set_ylabel(r"$\vert A_{sk}\vert^2$")
        axs4[i].set_ylabel(r"$\vert \Pi_{sk}\vert^2$")

    title = rf"$\alpha_\mathrm{{G}} = {alpha_val}$, $k_{{s}}$ = {pem_gauss.get_k():.0f}"
    axs1[i].set_title(title)
    axs2[i].set_title(title)
    axs3[i].set_title(title)
    axs4[i].set_title(title)

    if i == 0:
        axs1[i].legend(loc="best", ncols=2, fontsize="small")
        axs2[i].legend(loc="best", ncols=2, fontsize="small")
        axs3[i].legend(loc="best", ncols=2, fontsize="small")
        axs4[i].legend(loc="best", ncols=2, fontsize="small")
    
    axs1[i].set_xlim(None,zoom_start)
    axs2[i].set_xlim(None,zoom_start)
    axs3[i].set_xlim(None,zoom_start)
    axs4[i].set_xlim(None,zoom_start)
    
    axs1[i].grid(linewidth=0.2)
    axs2[i].grid(linewidth=0.2)
    axs3[i].grid(linewidth=0.2)
    axs4[i].grid(linewidth=0.2)
    
    print(f"Inset {i} y-limits after set_ylim: {inset_ax1.get_ylim()}")
    
    inset_ax1.grid(linewidth=0.2)
    inset_ax1b.grid(linewidth=0.2)
    inset_ax2.grid(linewidth=0.2)
    inset_ax2b.grid(linewidth=0.2)
    inset_ax3.grid(linewidth=0.2)
    inset_ax3b.grid(linewidth=0.2)
    inset_ax4.grid(linewidth=0.2)
    inset_ax4b.grid(linewidth=0.2)
    
    inset_ax1.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax1b.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax2.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax2b.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax3.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax3b.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax4.tick_params(left=False, right=True, labelleft=False, labelright=True)
    inset_ax4b.tick_params(left=False, right=True, labelleft=False, labelright=True)

    if i == 1:
        axs1[i].tick_params(left=False, right=False, labelleft=False, labelright=False)
        axs2[i].tick_params(left=False, right=False, labelleft=False, labelright=False)
        axs3[i].tick_params(left=False, right=False, labelleft=False, labelright=False)
        axs4[i].tick_params(left=False, right=False, labelleft=False, labelright=False)
    

# fig_basename = f"{figures_dir}/{currentset}_t{tau_f}"

# fig1.savefig(f"{fig_basename}_PE.pdf", bbox_inches="tight")
# fig2.savefig(f"{fig_basename}_PB.pdf", bbox_inches="tight")
# fig3.savefig(f"{fig_basename}_J11.pdf", bbox_inches="tight")
# fig4.savefig(f"{fig_basename}_J22.pdf", bbox_inches="tight")

plt.show()

In [ ]:
set_rc_params_article(ncol=1)
fig1, ax1 = plt.subplots()
axs = [None, ax1, None, None]

# compute_and_plot_PE_PB(pem_none, Vexp_none, tau_i_none, "none", axs)

for beta in np.geomspace(1.0e-1, 3.0, 1):
    for alpha in np.linspace(14.4, 14.4, 1):
        Vexp_cauchy.set_properties(betaem=beta, alphaem=alpha)
        Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)

        # compute_and_plot_PE_PB(
        #    pem_cauchy,
        #    Vexp_cauchy,
        #    tau_i_cauchy,
        #    rf"cauchy $\beta = {beta:.2g}$, $\alpha = {alpha}$, ",
        #    axs,
        # )
        compute_and_plot_PE_PB(
            pem_gauss,
            Vexp_gauss,
            rf"$\beta_\mathrm{{G}} = {beta:.1g}$, $\alpha_\mathrm{{G}} = {alpha}$, ",
            axs,
        )

ax1.legend(loc="best", ncol=2)
ax1.set_yscale("log")
ax1.set_xlabel(rf"$\tau$")
ax1.set_ylabel(rf"$P_{{B}}$ [nG]")
ax1.set_xlim(9.2, tau_f)
ax1.set_ylim(1.0e-9, 1.0e5)
ax1.yaxis.tick_right()

In [ ]:
set_rc_params_article(ncol=2)
fig1, ax1 = plt.subplots()
axs = [None, ax1, None, None]

# compute_and_plot_PE_PB(pem_none, Vexp_none, tau_i_none, "none", axs)

for beta in np.geomspace(2.98, 2.98, 1):
    for alpha in np.linspace(14.0, 15.2, 6):
        Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)
        tau_gauss_a, PE_gauss_a, PB_gauss_a, J11_gauss_a, J22_gauss_a = compute_PE_PB(
            pem_gauss, Vexp_gauss
        )
        ax1.plot(
            tau_gauss_a,
            np.sqrt(PB_gauss_a),
            lw=1,
            label=rf"$\alpha_\mathrm{{G}} = {alpha:.1f}$",
        )

ax1.set_title(r"$\beta_\mathrm{G}$ = 2.98")
ax1.legend(loc="best", ncol=2)
ax1.set_yscale("log")
ax1.set_xlabel(rf"$\tau$")
ax1.set_ylabel(rf"$B_{{\lambda}}$ [nG]")
ax1.set_xlim(9.2, 9.299)
ax1.set_ylim(1.0e-6, 1.0e1)
ax1.grid()
ax1.yaxis.tick_right()

plt.savefig(
    f"{figures_dir}/{currentset}_alphacloseup_beta2.98.pdf", bbox_inches="tight"
)

In [ ]:
alpha_min, alpha_max, nalphas = 12.8, 15.2, 30
beta_min, beta_max, nbetas = 1.0, 3.2, 30

alpha_vals = np.linspace(alpha_min, alpha_max, nalphas)
beta_vals = np.geomspace(beta_min, beta_max, nbetas)

Bmax = np.array(
    [
        [
            np.mean(compute_f_E_B(pem_gauss, Vexp_gauss, alpha, beta)[-10:])
            for alpha in alpha_vals
        ]
        for beta in beta_vals
    ]
)

#log10Bmax_interp = RegularGridInterpolator(
#    (alpha_vals, beta_vals), np.log10(Bmax), method="cubic"
#)

In [ ]:
log10Bmax_interp = RegularGridInterpolator(
    (alpha_vals, beta_vals), np.log10(Bmax)
)

In [ ]:
set_rc_params_article(ncol=1)

fine_grid_size = 1000
alpha_fine = np.linspace(alpha_min, alpha_max, fine_grid_size)
beta_fine = np.geomspace(beta_min, beta_max, fine_grid_size)
Bmax_fine_args = [[(alpha, beta) for alpha in alpha_fine] for beta in beta_fine]
log10Bmax_fine = log10Bmax_interp(Bmax_fine_args)

plt.contourf(
    beta_fine,
    alpha_fine,
    log10Bmax_fine,
    levels=[-12.0, -8.0, 0.0],
    colors=["blue","orange"],
    alpha=0.2,
)
contour_lines = plt.contour(
    beta_fine,
    alpha_fine,
    log10Bmax_fine,
    levels=[-12.0, -8.0, 0.0],
    colors=[ "blue","orange", "green"],
    linewidths=1.5,
)

plt.xlabel(r"$\beta_\mathrm{G}$")
plt.ylabel(r"$\alpha_\mathrm{G}$")
plt.ylim(12.0, None)

handles, labels = contour_lines.legend_elements()
plt.legend(
    handles,
    [rf"$B_{{\lambda}}=10^{{{level:.0f}}}$ \,nG" for level in contour_lines.levels],
    loc="lower left",
    ncol=1,
)

plt.savefig(
    f"{figures_dir}/{currentset}_t{tau_f}_alpha_beta_limits.pdf",
    bbox_inches="tight",
)
plt.show()

In [ ]:
ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 100)
k_a_plot = np.geomspace(ki, kf, 1000)
tau_ad_a = np.linspace(tau_f - 0.1, tau_f, 6)

print(
    f"Preparing spectrum with alpha = {Vexp_gauss.props.alphaem}, "
    f"beta = {Vexp_gauss.props.betaem}"
)
print(f"Inital time: {tau_ad_a[0]:.2f}, mode: {ki}")
ps_E_gauss, ps_B_gauss = pem_gauss.prepare_spectrum(Vexp_gauss, k_a, tau_ad_a)
print(f"Magnetic field: {ps_B_gauss.eval(Vexp_gauss, tau_ad_a[0], ki):.2e} nG")

In [ ]:
nlin = 3
beta_start_lin = 1.0
beta_end_lin = 2.8

nlog = 3
beta_start_log = beta_end_lin + 0.05
beta_end_log = 2.99

for alpha in [14.24, 14.4]:
    set_rc_params_article(ncol=1)
    plt.figure()

    n = nlin + nlog

    colorslin = plt.cm.winter(np.linspace(0, 1, nlin))
    colorslog = plt.cm.autumn(np.linspace(0, 0.6, nlog))
    ilin = 0
    ilog = 0

    for beta in np.linspace(beta_start_lin, beta_end_lin, nlin):
        Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)
        ps_E_gauss, ps_B_gauss = pem_gauss.prepare_spectrum(Vexp_gauss, k_a, tau_ad_a)
        print(f"Computing: alpha = {alpha:.2f}, beta = {beta:.2f}")
        color = colorslin[ilin]
        for tau in tau_ad_a[-1:]:
            ps_B_gauss_tau = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k_a_plot]
            plt.plot(
                k_a_plot,
                np.sqrt(ps_B_gauss_tau),
                label=rf"$\beta_\mathrm{{G}}=$ {beta:.2f}",
                color=color,
            )
        ilin += 1

    for beta in np.geomspace(beta_start_log, beta_end_log, nlog):
        Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)
        ps_E_gauss, ps_B_gauss = pem_gauss.prepare_spectrum(Vexp_gauss, k_a, tau_ad_a)
        print(f"Computing: alpha = {alpha:.2f}, beta = {beta:.2f}")
        color = colorslog[ilog]
        for tau in tau_ad_a[-1:]:
            ps_B_gauss_tau = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k_a_plot]
            plt.plot(
                k_a_plot,
                np.sqrt(ps_B_gauss_tau),
                label=rf"$\beta_\mathrm{{G}}=$ {beta:.2f}",
                linestyle="dashed",
                color=color,
            )
        ilog += 1

    plt.axhline(y=1.0, color="g", linestyle="-")
    plt.axhline(y=1.0e-8, color="orange", linestyle="-")
    plt.axhline(y=1.0e-12, color="b", linestyle="-")
    plt.text(10, 0.2, "CMB, UHECR, 21-cm...", va="center", ha="center", color="g")
    plt.text(
        90,
        4.0e-8,
        r"$\gamma$-rays",
        va="center",
        ha="center",
        color="orange",
        weight="bold",
    )
    plt.text(200, 7.0e-12, "dynamo", va="center", ha="center", color="b")
    plt.axhspan(1.0e-8, 1.0, facecolor="orange", alpha=0.1)
    plt.axhspan(1.0e-8, 1.0e-12, facecolor="blue", alpha=0.2)
    plt.xlabel(r"$k$")
    plt.ylabel(r"$B_{{\lambda,0}}(k)$")
    plt.xscale("log")
    plt.yscale("log")
    plt.grid()
    # plt.ylim(1.0e-12,1.0e1)
    plt.title(rf"$\alpha = {alpha:.2f}$")
    plt.legend(loc="lower right", ncol=2, fontsize="small")
    plt.savefig(
        f"{figures_dir}/{currentset}_t{tau_f}_B_k_{Vexp_gauss.props.alphaem:.2f}.pdf",
        bbox_inches="tight",
    )
    plt.show()

In [ ]:
set_rc_params_article(ncol=1)
plt.figure()

ki1 = 1.0
kf1 = 1.0e1
k1_a_plot = np.geomspace(ki1, kf1, 500)

ki2 = 6.0e1
kf2 = 4.0e3
k2_a_plot = np.geomspace(ki2, kf2, 500)

nlin = 0
beta_start_lin = 1.0
beta_end_lin = 2.8

nlin2 = 3
beta_start_lin2 = beta_end_lin + 0.05
beta_end_lin2 = 2.99

n = nlin + nlin2
colorslin = plt.cm.rainbow(np.linspace(0, 1, n))
i = 0

alpha = 14.4

for beta in np.linspace(beta_start_lin, beta_end_lin, nlin):
    Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)
    ps_E_gauss, ps_B_gauss = pem_gauss.prepare_spectrum(Vexp_gauss, k_a, tau_ad_a)
    print(Vexp_gauss.props.alphaem, Vexp_gauss.props.betaem)

    for tau in tau_ad_a[-1:]:
        ps_B_gauss_tau_slope = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k_a_plot]
        ps_B_gauss_tau_slope1 = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k1_a_plot]
        ps_B_gauss_tau_slope2 = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k2_a_plot]
        plt.plot(
            k_a_plot,
            np.sqrt(ps_B_gauss_tau_slope),
            label=rf"$\beta_\mathrm{{G}}=$ {beta:.3}",
            color=colorslin[i],
        )

    nB1 = np.log10(
        np.sqrt(ps_B_gauss_tau_slope1)[-1] / np.sqrt(ps_B_gauss_tau_slope1)[0]
    ) / np.log10(k1_a_plot[-1] / k1_a_plot[0])
    nB2 = np.log10(
        np.sqrt(ps_B_gauss_tau_slope2)[-1] / np.sqrt(ps_B_gauss_tau_slope2)[0]
    ) / np.log10(k2_a_plot[-1] / k2_a_plot[0])
    print("nB1: ", nB1, " nB2: ", nB2)

    plt.plot(
        k1_a_plot,
        np.sqrt(ps_B_gauss_tau_slope1[0]) * (k1_a_plot / k1_a_plot[0]) ** nB1,
        label=rf"$n_{{B1}}:$ {nB1:.3}",
        color=colorslin[i],
    )
    plt.plot(
        k2_a_plot,
        np.sqrt(ps_B_gauss_tau_slope2[0]) * (k2_a_plot / k2_a_plot[0]) ** nB2,
        label=rf"$n_{{B2}}:$ {nB2:.3}",
        color=colorslin[i],
    )
    i += 1


for beta in np.linspace(beta_start_lin2, beta_end_lin2, nlin2):
    Vexp_gauss.set_properties(betaem=beta, alphaem=alpha)
    ps_E_gauss, ps_B_gauss = pem_gauss.prepare_spectrum(Vexp_gauss, k_a, tau_ad_a)
    print(Vexp_gauss.props.alphaem, Vexp_gauss.props.betaem)

    for tau in tau_ad_a[-1:]:
        ps_B_gauss_tau_slope = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k_a_plot]
        ps_B_gauss_tau_slope1 = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k1_a_plot]
        ps_B_gauss_tau_slope2 = [ps_B_gauss.eval(Vexp_gauss, tau, k) for k in k2_a_plot]
        plt.plot(
            k_a_plot,
            np.sqrt(ps_B_gauss_tau_slope),
            label=rf"$\beta_\mathrm{{G}}=$ {beta:.3}",
            color=colorslin[i],
        )

    nB1 = np.log10(
        np.sqrt(ps_B_gauss_tau_slope1)[-1] / np.sqrt(ps_B_gauss_tau_slope1)[0]
    ) / np.log10(k1_a_plot[-1] / k1_a_plot[0])
    nB2 = np.log10(
        np.sqrt(ps_B_gauss_tau_slope2)[-1] / np.sqrt(ps_B_gauss_tau_slope2)[0]
    ) / np.log10(k2_a_plot[-1] / k2_a_plot[0])
    print("nB1: ", nB1, " nB2: ", nB2)

    plt.plot(
        k1_a_plot,
        np.sqrt(ps_B_gauss_tau_slope1[0]) * (k1_a_plot / k1_a_plot[0]) ** nB1,
        label=rf"$n_{{B1}}=$ {nB1:.3}",
        color=colorslin[i],
    )
    plt.plot(
        k2_a_plot,
        np.sqrt(ps_B_gauss_tau_slope2[0]) * (k2_a_plot / k2_a_plot[0]) ** nB2,
        label=rf"$n_{{B2}}=$ {nB2:.3}",
        color=colorslin[i],
    )
    i += 1

plt.xscale("log")
plt.yscale("log")
plt.axvline(x=ki1, color="g", linestyle="-")
plt.axvline(x=kf1, color="g", linestyle="-")
plt.axvline(x=ki2, color="g", linestyle="-")
plt.axvline(x=ki2, color="g", linestyle="-")
plt.legend(loc="best", ncol=2, fontsize="small")
plt.savefig(f"{figures_dir}/{currentset}_nB_smallbeta.pdf", bbox_inches="tight")
plt.show()